# Using fhir-query

## fq (fhir-query): Your FHIR Querying Assistant
The fq utility, short for "fhir-query," is a command-line tool specifically designed to simplify the process of interacting with FHIR servers. It provides researchers with a convenient way to:

1. Retrieve the vocabulary of a FHIR server: With the vocabulary command, fq fetches and summarizes the key data elements (CodeableConcepts and Extensions) used within the FHIR data. This creates a central vocabulary Dataframe that helps researchers identify important data elements and their usage within the server.


2. Execute queries to retrieve FHIR resources: Researchers can then use fq to execute FHIR queries using a readable syntax. This helps to retrieve and filter data from the FHIR Server based on various search parameters and criteria.

In [1]:
!pip install fhir-aggregator-client==0.1.8 --no-cache-dir --quiet

### Verify installation

In [2]:
!fq

Usage: fq [OPTIONS] COMMAND [ARGS]...

  FHIR-Aggregator utilities.

Options:
  --help  Show this message and exit.

Commands:
  ls          List all the installed GraphDefinitions.
  run         Run GraphDefinition queries.
  results     Work with the results of a GraphDefinition query.
  vocabulary  FHIR-Aggregator's key Resources and CodeSystems.


### Overview  

This notebook leverages **[FHIR GraphDefinition](https://hl7.org/fhir/graphdefinition.html)** objects to define and execute graph-based traversals across multiple interconnected FHIR resource graphs. The data retrieved is written to a **local SQLite database** for persistence and later transformed into **analyst-friendly dataframes** for analysis using tools like Python’s pandas library.

By using **FHIR GraphDefinition**, we declaratively define resource relationships and efficiently retrieve data. Once retrieved, the data is stored locally and can be transformed into dataframes for advanced analysis.

The [fhir-aggregator-client](https://github.com/FHIR-Aggregator/fhir-aggregator-client) tool runs an R5 GraphDefinition against a FHIR server

### Key Features  

- **GraphDefinition-Driven Traversals**: Use GraphDefinition objects to define explicit relationships between resources and automate traversal logic.  
- **Local SQLite Storage**: Persist the retrieved FHIR data in a local SQLite database for querying and offline analysis.  
- **Analyst-Friendly Dataframes**: Convert stored FHIR resources into pandas dataframes for ease of use in analytical workflows.  
- **Reusable Graph Definitions**: Maintain a library of GraphDefinition YAML files that can be reused across different workflows and projects.  Researchers and Data submitters can publish GraphDefinition files to help others navigate their data.

### List installed GraphDefinition files

In [3]:
!fq ls

| id                          | description                                                                                                                             |
|-----------------------------+-----------------------------------------------------------------------------------------------------------------------------------------|
| condition-graph             | (FHIR-Aggregator) Condition to ResearchStudy and children. fhir-query '/Condition?code:text=cholangiocarcinoma'                         |
| patient-survival-graph      | (FHIR-Aggregator) Retrieve Patient and Observations [NCIT_C156418,NCIT_C156419]. fhir-query '/ResearchStudy?identifier=TCGA-BRCA'       |
| research-study-link-iterate | (dbGAP) Retrieve ResearchStudy and children. Uses HAPI's deep linking. fhir-query '/ResearchStudy?_id=phs001232'                        |
| research-study-part-of      | (FHIR-Aggregator) Retrieve a ResearchStudy and children. Uses part-of-study extension. fhir-query '/ResearchStudy?iden

### Run a GraphDefinition

In [4]:
%env FHIR_BASE= https://google-fhir.fhir-aggregator.org

env: FHIR_BASE=https://google-fhir.fhir-aggregator.org


In [5]:
!fq run condition-graph /Condition?code:text=cholangiocarcinoma

condition-graph is valid FHIR R5 GraphDefinition

¡ Fetching https://google-fhir.fhir-aggregator.org/Condition?code:text=cholangiocarcinoma

Error: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Basic Constraints of CA cert not marked critical (_ssl.c:1028)



#### Analyse Results

The graph represents relationships between different FHIR resources.Examples of FHIR resources include Patient, Condition, Observation, Procedure, etc.

Each node is labled as: <resource_type\>/<count\> the number of records of that type retrieved.

The edges in the graph are weighted.  The thicker the line, the more connections there are between nodes.

In [6]:
# Create a graph of the results
!fq results visualize

ERROR:root:Error: 'charmap' codec can't encode characters in position 263607-263621: character maps to <undefined>
Traceback (most recent call last):
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\fhir_aggregator_client\cli.py", line 273, in visualize
    visualize_aggregation(db.aggregate(ignored_edges), output_path.name)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\fhir_aggregator_client\visualizer.py", line 34, in visualize_aggregation
    net.save_graph(str(output_path))
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyvis\network.py", line 438, in save_graph
    self.write_html(name)
    ~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\Parker.Gray\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyvis\network.py", line 530, in write_html
    out.write(se

In [26]:
# Read the locally stored HTML file containing a graph visualization and displaying it within the Jupyter notebook.

from IPython.display import HTML
with open('fhir-graph.html', 'r') as file:
    html_content = file.read()

# Set the display height (in pixels)
display(HTML("<div style='height: 800px;'>{}</div>".format(html_content)))

#### Create a dataframe of results

In [27]:
!fq results dataframe

Saved fhir-graph.tsv


In [28]:
import pandas as pd 
df = pd.read_csv('fhir-graph.tsv', sep='\t')
df

EmptyDataError: No columns to parse from file

### Other servers

You can use the `fq` tool with other FHIR servers.  For example, this query retrieves a study from `dbGAP`.

In [ ]:
# delete the previous results, start with a fresh database
!rm ~/.fhir-aggregator/fhir-graph.sqlite
!fq run  --fhir-base-url https://dbgap-api.ncbi.nlm.nih.gov/fhir-jpa-pilot/x1  research-study-link-iterate  '/ResearchStudy?_id=phs001232'

In [ ]:
# use the same commands to analyse results
!fq results visualize

In [ ]:
# create a graph of the results

from IPython.display import HTML
with open('fhir-graph.html', 'r') as file:
    html_content = file.read()

# Set the display height (in pixels)
display(HTML("<div style='height: 800px;'>{}</div>".format(html_content)))

In [ ]:
# create a dataframe of results
!fq results dataframe

In [ ]:
import pandas as pd

df = pd.read_csv('fhir-graph.tsv', sep='\t')

df